Questions:
- Separate am and pm into two dataframes?  (Both individual df's and summary df).

To do:


Steps:
- Remove duplicates (dupe check)
- Remove test cases (remove_dupes)
- Redo timestamps (time_stamp)
- Rename columns.
- Pull out array values.
- Split time values. Create "mean time"
- Process "selected" for evening df's.


In [1]:
import numpy as np, pandas as pd, os, re, time
os.chdir('/home/wraikes/Dropbox/partnership/dmt/data/sage_final')

In [38]:
def am_pm_data(files):
    for file in files:
        df = pd.read_csv(file)
        name = create_df_name(file)
        
        if 'metadata.json.taskIdentifier' in df.columns:

            df = dupe_check(df, name)
            df = remove_dupes(df)
            df = df[(df['metadata.json.taskIdentifier'] == 'am_survey') | 
                    (df['metadata.json.taskIdentifier'] == 'pm_survey')]
        
            if df.shape[0] > 0: 
                print(file)
                df = time_stamp(df)
                df = col_relabel(df, name)
                df = discount(df)
                
                if 'behavior' in name:
                    df = selected_spread(df)
                    
                if 'bart' in name:
                    df = std_change(df)

                df.to_csv('../sage_am_pm/am_pm_filter/' + file, index=False)

In [3]:
def create_df_name(string):
    string = string.replace('digital-marshmallow-', '')
    string = string.replace('.csv', '')
    
    return string 

In [4]:
def remove_dupes(df):
    
    test_users = [
        'ThpMV2Achc', 
        'SEkQVTCe6j', 
        'Wh8NSX3DHL', 
        'SaXFr2kPZa', 
        'VWUcSp4TeH', 
        'yXEfAmW682', 
        'gwEpQR8j9B',
        'WbbNWM4RAF', 
        'D5bzYrfd8E', 
        'LJcmEFWp74', 
        'ULoF3MM1nN',
        'G9Bbsg'
    ]

    diff_study = [
        'rL8eA3',
        'rLg5xs',
        'rLrD9h',
        'rLP7H2',
        'rL6s6h',
        'aOyzBg',
        'aORA43',
        'aOh48U',
        'aOLu4K',
        'aOQtxv',
        'aO5TvQ',
        'mPC9S8',
        'mPgquX',
        'mP5xkB',
        'mPSQvh',
        'mPYk2p',
        'mP3rbd'
    ]
  
    return df[~df.externalId.isin(test_users + diff_study)]

In [5]:
def time_stamp(df):
    
    for col in df.columns:
        if any([col.endswith(x) for x in ('createdOn', 'startDate', 'endDate')]):
            df[col] = df[col][df[col].notnull()].apply(lambda x: time.strftime("%m.%d.%Y %H:%M - %Z", time.localtime(x / 1000)))
    
    return df

In [6]:
def dupe_check(df, name):
    
    if name == 'goNoGo-v2':
        df = clean_df_dupe(df, 'ksJM3Y', -1)
        df = clean_df_dupe(df, 'S6b4eL', 11)
    
    elif name == 'delay_discounting_raw-v6':
        df = clean_df_dupe(df, 'S6b4eL', 2)
        
    elif name == 'discounting_raw-v2':
        df = clean_df_dupe(df, 'S6b4eL', 5)
    
    elif name == 'behavior_choices_4_bl-v2':
        df = clean_df_dupe(df, 'yQ7pYy', -1)
    
    return df


def clean_df_dupe(df, _id, pos):
   
    indices = df.index[df.externalId == _id]
    df.drop(indices[pos], axis=0, inplace=True)
    
    return df

In [7]:
def col_relabel(df, name):
    '''
    Remove: 'metadata.json.' and 'data.json.'.
    Replace spaces with underscores.
    Append new name to existing column name, except if externalId.
    '''
    new_cols = []
    col_re = re.compile('metadata.json.|data.json.')
    
    for col in df.columns:
        new_col = re.sub(col_re, '', col)
    
        if new_col != 'externalId':
            new_col = 'SAGE_{}___{}'.format(name, 
                                            new_col
                                            )
                                              
        new_col = new_col.replace(' ', '_')
        new_col = new_col.replace('-', '_')
        new_cols.append(new_col)
    
    df.columns = new_cols
    
    return df

In [8]:
def col_convert_dd(row, ix=None):
    '''
    Convert a column that contains a string of a list of strings, into a list of floats.
    
    Inputs
        row: a string of a list of strings.
        
    Output
        returns the list of floats
    
    '''
    if pd.notnull(row) and ix:
        return [float(x) for x in row.strip('[]').split(',')][ix]
    
    elif pd.notnull(row):
        return [float(x) for x in row.strip('[]').split(',')]
    
    else:
        return row
    
    
def array_split(value, i):
    if pd.notnull(value):
        return list(map(float, value[1:-1].split(',')))[i]
    else:
        return value

    
def discount(df):
    new_df = df.copy()
    
    for col in df.columns:
        #Create new column using "last" number in 'choiceArray'.
        if 'choiceArray' in col or 'nowArray' in col or 'variableArray' in col:
            col_name = col + '_last'
            new_df[col_name] = df[col].apply(lambda x: col_convert_dd(x, -1))   
    
        #Split times into new columns and times mean column.
        if 'times' in col:
            for i in range(6):
                new_col = col + '_v' + str(i)
                new_df[new_col] = df[col].apply(lambda x: array_split(x, i))
    
            col_name = col + '_mean'
            new_df[col_name] = df[col].apply(lambda x: np.mean(col_convert_dd(x)))
                
    return new_df

In [10]:
def selected_spread(df):
    #Note: codebook is incorrect. 'gamble' vs. 'gambling'.
    terms = ['junkfood', 'gambling', 'smoke', 'mobile', 'overeat', 'mast', 'cut', 'smedia',
             'alcohol', 'porn', 'hair', 'gaming', 'sitting', 'spend', 'temper', 'othermore']

    for term in terms:
        col_name = 'SAGE_morning_behavior_choices_v1___selected' + '_' + term
        df[col_name] = np.where(df['SAGE_morning_behavior_choices_v1___selected'].str.contains(term), 1, 0)
        
    return df

In [32]:
def col_convert_bart(row):
    '''
    Convert a column that contains a string of a list of strings, into a list of floats.  
    
    Inputs
        row: a string of a list of strings.
        
    Output
        returns a list of floats
    
    '''
    if pd.notnull(row):
        return [float(x) for x in row.strip('[]').split(',')]
    
    else:
        return row


def std_change(df):
    name = 'SAGE_bart_v4'

    stds = [
        '___pumps_standard_deviation',
        '___pumps_standard_deviation_first_third',
        '___pumps_standard_deviation_second_third',
        '___pumps_standard_deviation_last_third'
    ]

    df[name + stds[0]] = df[name + '___pumps_per_balloon'].apply(col_convert_bart).apply(np.std)
    df[name + stds[1]] = df[name + '___pumps_per_balloon'].apply(col_convert_bart).apply(lambda x: x if np.any(pd.isnull(x)) else x[:5]).apply(np.std)
    df[name + stds[2]] = df[name + '___pumps_per_balloon'].apply(col_convert_bart).apply(lambda x: x if np.any(pd.isnull(x)) else x[5:10]).apply(np.std)
    df[name + stds[3]] = df[name + '___pumps_per_balloon'].apply(col_convert_bart).apply(lambda x: x if np.any(pd.isnull(x)) else x[10:]).apply(np.std)
        
    return df

In [40]:
am_pm_data(os.listdir())

digital-marshmallow-goNoGo-v2.csv
digital-marshmallow-morning_yesterday_sem_diff-v2.csv
digital-marshmallow-morning_sem_diff-v2.csv
digital-marshmallow-pam-v2.csv
digital-marshmallow-delay_discounting_raw-v6.csv
digital-marshmallow-evening_sem_diff-v2.csv
digital-marshmallow-morning_behavior_choices-v1.csv
digital-marshmallow-discounting_raw-v2.csv
digital-marshmallow-morning_yesterday_likert-v2.csv
digital-marshmallow-bart-v4.csv
